# Fonction renvoyant une durée restée dans un lieu selon l'heure et le lieu

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import ast
from scipy.stats import invgamma,loggamma,invgauss

!pip install openpyxl

In [2]:
#Lire le tableau des lois de durée restée dans un lieu:
tableau_duree= pd.read_excel("data/lois_duree.xlsx")

In [3]:
tableau_duree=tableau_duree.set_index("Unnamed: 0").rename_axis("Plage horaire")

In [4]:
tableau_duree=tableau_duree.map(lambda x: ast.literal_eval(x))


In [5]:
tableau_duree.head()

,Rue,Entreprise,Sans,Domicile,Parking
Plage horaire,,,,,
Matin,"[invgauss, (4.036204719120207, -0.161617635419...","[beta, (52.595462995358545, 27.703319219454222...","[beta, (0.5117940197816365, 19.2230902247954, ...","[beta, (0.5474258078179828, 0.9000121427405545...","[invgamma, (1.2485500680449202, -0.23360330017..."
Midi,"[chi2, (1.3400808769713695, -5.127648667537803...","[invgamma, (54.707261432752674, -13.6736528297...","[beta, (0.30450496225678947, 88.52544141212059...","[beta, (0.547862902931137, 0.6163748511340661,...","[pareto, (3.474502202630147, -5.75279416070615..."
Après-midi,"[chi2, (1.3380657374339826, -2.022860775150514...","[gamma, (1.1060959455943804, -0.00407001505164...","[beta, (0.8611652542892512, 79.35796489004701,...","[beta, (1.8969335796940112, 1.1440186443623233...","[invgamma, (2.030103317853264, -0.273339157562..."
Soir,"[beta, (0.7222192851774487, 1.4655167671346607...","[beta, (0.8699981002941614, 1.3044457209426563...","[beta, (0.3021326639337568, 266.1616492102139,...","[beta, (1.9493353130904008, 1.1828119830488304...","[invgauss, (1.3223013343950307, -0.16386888664..."


In [10]:
def duree_lieu(heure_arrivee,lieu):
    if heure_arrivee<=11:
        plage= "Matin"
    elif heure_arrivee<=14:
        plage="Midi"
    elif heure_arrivee<=17:
        plage= "Après-midi"
    else:
        plage= "Soir"
    loi=tableau_duree.loc[plage,lieu]
    dist=loi[0]
    #print(dist)
    param=list(loi[1])
    if abs(param[-2])<10**(-2):
        param[-2]=0
    #print(param)
    sample = getattr(stats,dist).rvs(*param)
    if sample>=0:
        return sample
    else:
        print(sample)
        duree_lieu(heure_arrivee,lieu)

In [24]:
for i in range(200):
    print(duree_lieu(0,"Entreprise"))
    

4.408814705299804
9.65336237903287
7.100328156162313
11.123138350226625
5.6127108204842955
7.4777440615335
6.55372509495259
5.47487684821774
6.58823344258974
0.8810436801740877
5.177427199331177
8.84007295204065
13.455174451723103
12.501757717010761
10.526163859185445
8.444985618605827
8.547586259868929
9.958757501060006
5.5315173238565265
7.522499969034634
4.005447990252897
5.4882353576831875
7.245354369318925
4.0971857550989625
9.635910593097663
9.312737346730849
11.325363732063991
6.602597247336263
3.9358073790155075
12.153068629297508
-1.8887755961029669
None
11.263829060641832
5.986391265515444
7.866339313823289
7.926373183981319
13.552023183635995
5.150931166132686
5.811425634590407
3.1613635113197915
8.806398638300784
3.905714427422197
11.19685840993882
5.902127230172397
6.898989410118112
5.009604988664904
7.1889229967107156
11.731693726799072
2.404510173675874
0.39855434912285403
7.950454935414108
7.5124962626977165
4.737479974918784
9.038466267501668
4.934751448265189
7.468109